# Offline Batch Recommender System

In this notebook, we will build a simple offline batch recsys that writes results to Redis for later access. The architecture diagram below shows how this system comes together.

![](./img/OfflineBatchRecsys.png)

## Candidate Retrieval Model

Now about the model itself... many recommender systems have a *two-stage pipeline*:
1) A fast **candidate retrieval** model quickly truncates the large item catalog to a relevant set of hundreds (or thousands) of options
2) A finely-tuned **ranking model** (i.e. more powerful) ranks the most likely items that are going to interacted with.

In this notebook, we will build a simple **Two-Tower** candidate retrieval model with Tensorflow and Merlin/NVTabular helper utilities that can score millions of items for a given user. The Two-Tower model is a neural network architecture with two MLP towers where both user and item features are fed to generate user and item embeddings in the output.

Though we skip the ranking model step for now, you will pick that up in the [Multi-Stage Recommender System]() example notebook.

*This notebook was created using the latest stable [merlin-tensorflow](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/merlin/containers/merlin-tensorflow/tags) container and was heavily based on the work done by the NVIDIA Merlin team [here](https://github.com/NVIDIA-Merlin/models/blob/main/examples/05-Retrieval-Model.ipynb)*

## About the Dataset

In this notebook, we use a synthetic dataset that are mimicking the [Ali-CCP: Alibaba Click and Conversion Prediction](https://tianchi.aliyun.com/dataset/dataDetail?dataId=408#1) dataset. The synthetic nature allows us to tune it to our exact needs for demonstration/learning purposes.


### Importing Libraries

In [ ]:
import os
import logging

import nvtabular as nvt
import merlin.models.tf as mm
import tensorflow as tf

from nvtabular.ops import *

from merlin.datasets.synthetic import generate_data
from merlin.datasets.ecommerce import transform_aliccp
from merlin.models.utils.example_utils import workflow_fit_transform
from merlin.models.utils.dataset import unique_rows_by_features
from merlin.schema.tags import Tags
from merlin.io.dataset import Dataset


# disable INFO and DEBUG logging everywhere
logging.disable(logging.WARNING)

## Generate Synthetic Ali-CCP Dataset

In [ ]:
def generate_aliccp_data(num_rows: int, train_size: float, valid_size: float):
    train, valid = generate_data("aliccp-raw", num_rows, set_sizes=(train_size, calid_size))
    train = train.to_ddf().compute()
    valid = valid.to_ddf().compute()
    return train, valid


In [ ]:
# Generate the data

NUM_ROWS = 1000000
TRAIN_SIZE = 0.7
VALID_SIZE = 0.3

train, valid = generate_aliccp_data(NUM_ROWS, TRAIN_SIZE, VALID_SIZE)

In [ ]:
# Truncate datasets to only "positive" click interactions between User/Item pairs
train = train.loc[train['click']==1].reset_index(drop=True)
valid = valid.loc[valid['click']==1].reset_index(drop=True)

# Drop the "target" interaction fields -- no longer need them
train = train.drop(['click', 'conversion'], axis=1)
valid = valid.drop(['click', 'conversion'], axis=1)

**Note:** To be able to learn from this implicit feedback, we use the naive assumption that the interacted items are **more relevant** for the user than the non-interacted ones.

This is an assumption for simplification purposes so we can use a negative sampling technique. 

In [ ]:
# Use the Merlin Dataset wrapper to create dataset objects

train = Dataset(train)
valid = Dataset(valid)

In [ ]:
# Defin output path for data

DATA_DIR = os.environ['PWD'] +"data/"
OUTPUT_DATA_DIR = os.path.join(DATA_DIR, "processed")
CATEGORY_TEMP_DIR = os.path.join(DATA_DIR, "categories")


In [ ]:
# Define Feature Transformation Pipeline

user_id = ["user_id"] >> Categorify(out_path=CATEGORY_TEMP_DIR) >> TagAsUserID()
item_id = ["item_id"] >> Categorify(out_path=CATEGORY_TEMP_DIR) >> TagAsItemID()

item_features = ["item_category", "item_shop", "item_brand"] >> Categorify(out_path=CATEGORY_TEMP_DIR) >> TagAsItemFeatures()

user_features = (
    [
        "user_shops",
        "user_profile",
        "user_group",
        "user_gender",
        "user_age",
        "user_consumption_2",
        "user_is_occupied",
        "user_geography",
        "user_intentions",
        "user_brands",
        "user_categories",
    ]
    >> Categorify(out_path=CATEGORY_TEMP_DIR)
    >> TagAsUserFeatures()
)

outputs = user_id + item_id + item_features + user_features

With `transform_aliccp` function, we can execute fit() and transform() on the raw dataset applying the operators defined in the NVTabular workflow pipeline above. The processed parquet files are saved to output_path.

In [ ]:
# Transform data and create files
transform_aliccp((train, valid), OUTPUT_DATA_DIR, nvt_workflow=outputs)

## Building a Two-Tower Model

We will use Two-Tower Model to infer a subset of relevant items from large item corpus for a given user. 

A Two-Tower Model consists of item (candidate) and user (query) encoder towers. With two towers, the model can learn representations (embeddings) for queries and candidates separately. 

> NEED TO FIND IMG
<img src="./images/TwoTower.png"  width="30%">

Image Adapted from: [Off-policy Learning in Two-stage Recommender Systems](https://dl.acm.org/doi/abs/10.1145/3366423.3380130)

In [ ]:
# Load from file
train = Dataset(os.path.join(output_path, "train", "*.parquet"))
valid = Dataset(os.path.join(output_path, "valid", "*.parquet"))

Use the `schema` object to define our model. Select features with user and item tags, and be sure to exclude target column.

In [ ]:
# Schema will consist of the User ID, Item ID, User Features, and Item Features (as defined above)
schema = train.schema.select_by_tag([Tags.ITEM_ID, Tags.USER_ID, Tags.ITEM, Tags.USER])

# Set the schema for our datasets
train.schema = schema
valid.schema = schema

Inspect the column names in the schmea here:

In [ ]:
schema

As expected, we shouldn't have any label/target data yet

In [ ]:
label_names = schema.select_by_tag(Tags.TARGET).column_names
label_names

### About Negative Sampling

Many datasets for recommender systems contain implicit feedback with logs of user interactions like clicks, add-to-cart, purchases, music listening events, rather than explicit ratings that reflects user preferences over items. 


In Merlin Models -- NVIDIA provides some scalable negative sampling algorithms for this Item Retrieval task. In this example, we use the `in-batch` sampling algorithm which uses the items interacted by other users as negatives within the same mini-batch.

### Model Architecture

The **Two-Tower** model consists of a **User tower** (where all user features are fed) and an **Item tower** (where all item features are fed).

The User tower generates an embedding for the User. Then it computes the positive interaction "score" (likelihood of interaction event) using the dot-product between the User embedding and the Item embedding, in addition to sampled "negative" Items within a batch.

In [ ]:
def create_two_tower(tower_dim: int, encoder_dim: int, optimizer: str, k: int, tags) -> mm.TwoTowerModelV2:
    # User/Query Tower
    user_schema = schema.select_by_tag(tags.USER)
    # create user (query) tower input block
    user_inputs = mm.InputBlockV2(user_schema)
    # create user (query) encoder block
    query = mm.Encoder(
        user_inputs,
        mm.MLPBlock([encoder_dim, tower_dim], no_activation_last_layer=True)
    )

    # Item/Candidate Tower
    item_schema = schema.select_by_tag(tags.ITEM)
    # create item (candidate) tower input block
    item_inputs = mm.InputBlockV2(item_schema)
    # create item (candidate) encoder block
    candidate = mm.Encoder(
        item_inputs,
        mm.MLPBlock([encoder_dim, tower_dim], no_activation_last_layer=True)
    )
    
    # Build Model Class
    model = mm.TwoTowerModelV2(query, candidate)
    model.compile(optimizer=optimizer, run_eagerly=False, metrics=[mm.RecallAt(k), mm.NDCGAt(k)])
    return model

**Notes:**
- `no_activation_last_layer:` when set True, no activation is used for top hidden layer. Learn more [here](https://storage.googleapis.com/pub-tools-public-publication-data/pdf/b9f4e78a8830fe5afcf2f0452862fb3c0d6584ea.pdf).
- In the `TwoTowerModelV2` function we did not set `negative_samplers` arg. By default, it uses contrastive learning and `in-batch` negative sampling strategy.
- Two metrics are used to judge the quality of the recommendations: **Normalized Discounted Cumulative Gain (NDCG@K)** and **Recall@K**.
    - NDCG@K accounts for rank of the relevant item in the recommendation list and is a more fine-grained metric than HR, which only verifies whether the relevant item is among the top-k items.
    - Recall (Also known as HitRate@K) when there is only one relevant item in the recommendation list. Recall just verifies whether the relevant item is among the top-k items.
- When we set `validation_data=valid` in the `model.fit()`, we compute evaluation metrics on validation set using the negative sampling strategy used for training. 

In [ ]:
# Initialize model
model = create_two_tower(
    tower_dim=64,
    encoder_dim=128,
    optimizer="adam",
    k=10,
    tags=Tags
)

# Fit model
model.fit(train, validation_data=valid, batch_size=4096, epochs=2)

### Evaluate the model accuracy

The validation metric values during training are calculated given the positive and negative scores in each batch, and then averaged over batches per epoch. **That means validation metrics are not computed using the entire item catalog.**

To determine the exact accuracy, we need to compute the similarity score between a given query and all possible candidates. Below, by using the `topk_model` we can evaluate the trained retrieval model using the entire item catalog (brute force).

In [ ]:
# Create candidate/item features for evaluation
candidate_features = unique_rows_by_features(train, Tags.ITEM, Tags.ITEM_ID)
candidate_features.head()

In [ ]:
# Convert model to a top_k_encoder
topk_model = model.to_top_k_encoder(candidate_features, k=20, batch_size=128)

# we can set `metrics` param in the `compile(), if we want
topk_model.compile(run_eagerly=False)

In [ ]:
# Look at results?

In [ ]:
# Create data loader for validation data
eval_loader = mm.Loader(valid, batch_size=1024).map(mm.ToTarget(schema, "item_id"))

# Evaluation
metrics = topk_model.evaluate(eval_loader, return_dict=True)
metrics

### Generate top-K recommendations

We trained a model, now we can generate recommendations offline using `to_top_k_encoder` method. The `to_top_k_encoder()` uses the pre-trained candidate and query encoders to initialize a top-k encoder model, called as `topk_model` in this example. Practically, this method applies the candidate_encoder on the provided candidate_features dataset to set the top-k index of the `topk_model`. Therefore, topk_model object is the one responsible of generating the top-k predictions.

Let's generate top-K (k=20 in our example) recommendations for a given batch of 8 samples. The `to_top_k_encoder()` method uses the candidate (item) features dataset as the identifiers, i.e., we extract the`  candidate_id` arg using `Tags.ITEM_ID` tag by default and set it as `index` when calculating the candidate embeddings. The forward method of `topk_model` takes as the query features as input, and computes the dot product scores between the given query embeddings and all the candidates of the top-k index. Then, it returns the top-k (k=20) item ids with the highest scores. Note that instead of calculating the candidate (item) tower embeddings for each user query, we compute the output of the item tower once and store it in the `TopKEncoder` class  to use for the Top-k index. This is computationally more efficient.

In [ ]:
def generate_topk_recs(user_count: int, valid)
    eval_loader = mm.Loader(valid, batch_size=user_count, shuffle=False)
    for batch in loader
    batch = next(iter(eval_loader))
    print(batch[0]['user_id'])
    return topk_model(batch[0])

The recommended top 20 item ids are returned below for each of the 8 selected users (from the validation set). The output of the method is a named tuple `TopKPrediction`, where the first element is the dot product scores and the second element is the encoded item ids (not the original ids).

In [ ]:
generate_topk_recs(8, valid)

## Writing Recommendations to the Inference Store

Redis is used (low latency k-v store) to persist recommendations for each User.

### Simple Redis Intro

In [ ]:
import redis

redis_conn = redis.Redis(
    host="redis-inference-store",
    port=6379
)

# Simple HASH set
redis_conn.hset("test", mapping={"id": 1, "foo": "bar"})

In [ ]:
# Simple Hash get
redis_conn.hgetall("test")

In [ ]:
# Cleanup
redis_conn.delete("test")

In [ ]:
import asyncio
import redis.asyncio as redis

from redis.commands.json.path import Path


def generate_topk_recs(topk_model, K: int, batch_size: int):
    loader = mm.Loader(valid, batch_size=batch_size, shuffle=False)
    for batch in loader
        results = topk_model(batch[0])
        # TODO - need to inspect this result to figure out how to grab the top K
        # TODO figure the generator part out
        for user_id, res in results:
            yield user_id, res

async def store_recommendations(n: int, redis_conn: redis.Redis):
    """
    Store recommendations generated for each User.
    """
    semaphore = asyncio.Semaphore(n)
    async def store(user_id: str, recs: list):
        """
        Store and individual User's latest recommendations in Redis.
        """
        async with semaphore:
            entry = {
                "id": user_id,
                "recommendations": recommmendations
            }
            # Set the JSON object in Redis
            await redis_conn.json().set(f"USER:{user_id}", Path.root_path(), entry)
    
    # create generator
    topk_recs_per_user = generate_topk_recs(topk_model, K, batch_size)
    # gather with "concurrency"
    await asyncio.gather(*[store(user_id, recs) for user_id, recs in topk_recs_per_user])


In [ ]:
redis_conn = redis.Redis(
    host="redis-inference-store",
    port=6379
)

# Run the process
asyncio.run(store_recommendations(100, redis_conn, topk_model, K=10, batch_size=1024))

In [ ]:
# Check the results
redis_conn.dbsize()

## Exporting Retrieval Models

So far we have trained and evaluated our Retrieval model. Now, the next step is to deploy our model and generate top-K recommendations given a user (query). We can efficiently serve our model by indexing the trained item embeddings into an **Approximate Nearest Neighbors (ANN)** engine. Basically, for a given user query vector, that is generated passing the user features into user tower of retrieval model, we do an ANN search query to find the ids of nearby item vectors, and at serve time, we score user embeddings over all indexed top-K item embeddings within the ANN engine.

In doing so, we need to export
 
- user (query) tower
- item and user features
- item embeddings

#### Save and Load User (query) tower

We are able to save the user tower model as a TF model to disk. The user tower model is needed to generate a user embedding vector when a user feature vector <i>x</i> is fed into that model.

In [ ]:
query_tower = model.query_encoder
query_tower.save(os.path.join(DATA_FOLDER, "query_tower"))

## we can load back the saved model via the following script.
#query_tower_loaded = tf.keras.models.load_model(os.path.join(DATA_FOLDER, 'query_tower'))

#### Extract and save User features

With `unique_rows_by_features` utility function we can easily extract both unique user and item features tables as cuDF dataframes. Note that for user features table, we use `USER` and `USER_ID` tags.

In [ ]:
user_features = (
    unique_rows_by_features(train, Tags.USER, Tags.USER_ID).compute().reset_index(drop=True)
)

In [ ]:
user_features.head()

In [ ]:
# save to disk
user_features.to_parquet(os.path.join(DATA_FOLDER, "user_features.parquet"))

#### Generate Query embeddings for entire user catalog

In [ ]:
queries = model.query_embeddings(Dataset(user_features, schema=schema), batch_size=1024, index=Tags.USER_ID)
query_embs_df = queries.compute(scheduler="synchronous").reset_index()

In [ ]:
query_embs_df.head()

#### Extract and save Item features

In [ ]:
item_features = (
    unique_rows_by_features(train, Tags.ITEM, Tags.ITEM_ID).compute().reset_index(drop=True)
)

In [ ]:
item_features.head()

In [ ]:
# save to disk
item_features.to_parquet(os.path.join(DATA_FOLDER, "item_features.parquet"))

#### Extract and save Item embeddings

In [ ]:
item_embs = model.candidate_embeddings(Dataset(item_features, schema=schema), batch_size=1024, index=Tags.ITEM_ID)

In [ ]:
item_embs_df = item_embs.compute(scheduler="synchronous")

In [ ]:
item_embs_df

In [ ]:
# save to disk
item_embs_df.to_parquet(os.path.join(DATA_FOLDER, "item_embeddings.parquet"))

That's it. You have learned how to train and evaluate your Two-Tower retrieval model, and then how to export the required components to be able to deploy this model to generate recommendations. In order to learn more on serving a model to [Triton Inference Server](https://github.com/triton-inference-server/server), please explore the examples in the [Merlin](https://github.com/NVIDIA-Merlin/Merlin) and [Merlin Systems](https://github.com/NVIDIA-Merlin/systems) repos.